In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [6]:
df_=pd.read_excel("english_data.xlsx",sheet_name="Year 2009-2010")

In [51]:
df=df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [52]:
df["TotalPrice"]=df["Quantity"]*df["Price"]
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,5.90
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,7.50


In [53]:
df.groupby("Invoice").agg({"TotalPrice":"sum"})

,TotalPrice
Invoice,
489434,505.30
489435,145.80
489436,630.33
489437,310.75
489438,2286.24
...,...
C538121,-12.75
C538122,-1.25
C538123,-7.50


In [54]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

In [55]:
df.dropna(inplace=True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
TotalPrice     0
dtype: int64

In [56]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.0,12.758815,101.220424,-9360.00,2.00,4.00,12.00,19152.00
Price,417534.0,3.887547,71.131797,0.00,1.25,1.95,3.75,25111.09
Customer ID,417534.0,15360.645478,1680.811316,12346.00,13983.00,15311.00,16799.00,18287.00
TotalPrice,417534.0,19.994081,99.915863,-25111.09,4.25,11.25,19.35,15818.40


In [57]:
df=df[~df["Invoice"].str.contains("C",na=False)]

In [58]:
#r f m metriklerini oluşturma
today=dt.datetime(2010,12,11)
rfm=df.groupby("Customer ID").agg({"InvoiceDate":lambda InvoiceDate: (today-InvoiceDate.max()).days,
                                   "Invoice" :lambda Invoice :Invoice.nunique(),
                                   "TotalPrice":"sum"})
rfm.columns=["recency","frequency","monetary"]

In [59]:
rfm

,recency,frequency,monetary
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93
...,...,...,...
18283.0,18,6,641.77
18284.0,67,1,461.68
18285.0,296,1,427.00


In [60]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.0,91.269819,96.944304,1.0,18.00,53.00,136.0000,374.00
frequency,4314.0,4.454103,8.168658,1.0,1.00,2.00,5.0000,205.00
monetary,4314.0,2047.288659,8912.523243,0.0,307.95,705.55,1722.8025,349164.35


In [61]:
rfm=rfm[rfm["monetary"]>0]
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.0,91.172542,96.861457,1.00,18.0000,53.00,136.0000,374.00
frequency,4312.0,4.455705,8.170213,1.00,1.0000,2.00,5.0000,205.00
monetary,4312.0,2048.238236,8914.481280,2.95,307.9875,706.02,1723.1425,349164.35


In [62]:
rfm["monetary_score"]=pd.qcut(rfm["monetary"],5,labels=[1,2,3,4,5])
rfm

C:\Users\cafer\AppData\Local\Temp\ipykernel_15824\784960976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["monetary_score"]=pd.qcut(rfm["monetary"],5,labels=[1,2,3,4,5])


,recency,frequency,monetary,monetary_score
Customer ID,,,,
12346.0,165,11,372.86,2
12347.0,3,2,1323.32,4
12348.0,74,1,222.16,1
12349.0,43,3,2671.14,5
12351.0,11,1,300.93,2
...,...,...,...,...
18283.0,18,6,641.77,3
18284.0,67,1,461.68,2
18285.0,296,1,427.00,2


In [63]:
rfm["frequency_score"]=pd.qcut(rfm["frequency"].rank(method="first"),5,labels=[1,2,3,4,5])
rfm

C:\Users\cafer\AppData\Local\Temp\ipykernel_15824\3215910358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["frequency_score"]=pd.qcut(rfm["frequency"].rank(method="first"),5,labels=[1,2,3,4,5])


,recency,frequency,monetary,monetary_score,frequency_score
Customer ID,,,,,
12346.0,165,11,372.86,2,5
12347.0,3,2,1323.32,4,2
12348.0,74,1,222.16,1,1
12349.0,43,3,2671.14,5,3
12351.0,11,1,300.93,2,1
...,...,...,...,...,...
18283.0,18,6,641.77,3,5
18284.0,67,1,461.68,2,2
18285.0,296,1,427.00,2,2


In [64]:
rfm["recency_score"]=pd.qcut(rfm["recency"],5,labels=[5,4,3,2,1])
rfm

C:\Users\cafer\AppData\Local\Temp\ipykernel_15824\570077558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["recency_score"]=pd.qcut(rfm["recency"],5,labels=[5,4,3,2,1])


,recency,frequency,monetary,monetary_score,frequency_score,recency_score
Customer ID,,,,,,
12346.0,165,11,372.86,2,5,2
12347.0,3,2,1323.32,4,2,5
12348.0,74,1,222.16,1,1,2
12349.0,43,3,2671.14,5,3,3
12351.0,11,1,300.93,2,1,5
...,...,...,...,...,...,...
18283.0,18,6,641.77,3,5,4
18284.0,67,1,461.68,2,2,3
18285.0,296,1,427.00,2,2,1


In [66]:
rfm["rank"]=rfm["recency_score"].astype(str)  + rfm["frequency_score"].astype(str)
rfm

C:\Users\cafer\AppData\Local\Temp\ipykernel_15824\1828609071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["rank"]=rfm["recency_score"].astype(str)  + rfm["frequency_score"].astype(str)


,recency,frequency,monetary,monetary_score,frequency_score,recency_score,rank
Customer ID,,,,,,,
12346.0,165,11,372.86,2,5,2,25
12347.0,3,2,1323.32,4,2,5,52
12348.0,74,1,222.16,1,1,2,21
12349.0,43,3,2671.14,5,3,3,33
12351.0,11,1,300.93,2,1,5,51
...,...,...,...,...,...,...,...
18283.0,18,6,641.77,3,5,4,45
18284.0,67,1,461.68,2,2,3,32
18285.0,296,1,427.00,2,2,1,12


In [67]:
rfm[rfm["rank"]=="55"]

,recency,frequency,monetary,monetary_score,frequency_score,recency_score,rank
Customer ID,,,,,,,
12415.0,11,7,19543.84,5,5,5,55
12431.0,9,13,4370.52,5,5,5,55
12471.0,10,49,20139.74,5,5,5,55
12472.0,5,13,11308.48,5,5,5,55
12474.0,14,13,5048.66,5,5,5,55
...,...,...,...,...,...,...,...
18225.0,1,15,7545.14,5,5,5,55
18226.0,14,15,6650.83,5,5,5,55
18229.0,2,10,3526.81,5,5,5,55


In [69]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['rank'].replace(seg_map, regex=True)

C:\Users\cafer\AppData\Local\Temp\ipykernel_15824\298428299.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['segment'] = rfm['rank'].replace(seg_map, regex=True)


In [70]:
rfm

,recency,frequency,monetary,monetary_score,frequency_score,recency_score,rank,segment
Customer ID,,,,,,,,
12346.0,165,11,372.86,2,5,2,25,cant_loose
12347.0,3,2,1323.32,4,2,5,52,potential_loyalists
12348.0,74,1,222.16,1,1,2,21,hibernating
12349.0,43,3,2671.14,5,3,3,33,need_attention
12351.0,11,1,300.93,2,1,5,51,new_customers
...,...,...,...,...,...,...,...,...
18283.0,18,6,641.77,3,5,4,45,loyal_customers
18284.0,67,1,461.68,2,2,3,32,about_to_sleep
18285.0,296,1,427.00,2,2,1,12,hibernating


In [75]:
rfm[["segment","recency","frequency","monetary"]].groupby("segment").agg(["mean","count"])

recency        frequency           monetary      
                           mean count       mean count         mean count
segment                                                                  
about_to_sleep        53.819242   343   1.201166   343   441.320000   343
at_Risk              152.158756   611   3.073650   611  1188.878316   611
cant_loose           124.116883    77   9.116883    77  4099.450000    77
champions              7.119155   663  12.553544   663  6852.264167   663
hibernating          213.885714  1015   1.126108  1015   403.977836  1015
loyal_customers       36.287062   742   6.830189   742  2746.067353   742
need_attention        53.265700   207   2.449275   207  1060.357005   207
new_customers          8.580000    50   1.000000    50   386.199200    50
potential_loyalists   18.793037   517   2.017408   517   729.510986   517
promising             25.747126    87   1.000000    87   367.086782    87